Ce fichier a servi pour construire le script extractor.py

In [2]:
import pandas as pd
import sqlite3

In [3]:
database = sqlite3.connect("/home/irtimid/Documents/travail/leem/experiences/20150112_stefanoFarolfi_CPRInformation/data.sqlite")

In [40]:
partstemp = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", database)
parts = [p for p in partstemp.name]
parts

[u'parties',
 u'sessions',
 u'joueurs',
 u'partie_base',
 u'partie_revVolInfoCprREP',
 u'partie_questionnaireFinal',
 u'partie_revVolInfoCprOS',
 u'parties_joueurs__joueurs_parties',
 u'partie_revVolInfoCprREP_repetitions']

In [41]:
repeatedparts = [p for p in parts if "repetitions" in p]
repeatedparts    

[u'partie_revVolInfoCprREP_repetitions']

In [74]:
filterperso = lambda x: "partie_" in x and "base" not in x and "repetitions" not in x
oneshotparts = [p for p in parts if filterperso(p) and 
               not any(q.find(p) != -1 for q in repeatedparts)]
#toremove = [p for p in oneshotparts if any(q.find(p) != -1 for q in repeatedparts)]
#toremove
oneshotparts

[u'partie_questionnaireFinal', u'partie_revVolInfoCprOS']

In [65]:
repeatedparts.append("partie_bienPublic_repetitions")
filterperso = lambda p: 0 in [q.find(p) for q in repeatedparts]
for p in filter(filterperso, oneshotparts):
    oneshotparts.remove(p)
oneshotparts

[u'partie_questionnaireFinal', u'partie_revVolInfoCprOS']

In [4]:
parts2 = [
        "_".join(p.split("_")[1:]) for p in parts.name if
        "partie_" in p and "repetitions" not in p and
        "base" not in p
    ]
parts2

[u'revVolInfoCprREP', u'questionnaireFinal', u'revVolInfoCprOS']

In [5]:
sessions = pd.read_sql("select sessions.* from sessions where sessions.isTest = 0 order by sessions.nom", database)
sessions

,id,nom,debut,fin,isTest,parametres,commentaires
0,3,201502200955,09:55:25,11:07:14,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,None
1,4,201503041251,12:51:04,14:34:53,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,None
2,5,201503101257,12:57:08,14:32:48,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,None
3,6,201503180924,09:24:14,11:03:21,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,Erreur dans le oneshot c'est le traitement bas...
4,7,201505130918,09:18:11,10:55:11,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,None
5,8,201505191251,12:51:44,14:58:38,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,Interruption de la session à la période 12 sui...
6,9,201505271252,12:52:23,14:46:14,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,None
7,10,201505281256,12:56:14,14:37:24,0,PARTIE revVolInfoCprOS\n# -*- coding: utf -8 -...,None


In [6]:
base = pd.read_sql(
    "SELECT s.nom as session, j.uid as joueur, m.* " \
    "from sessions s, joueurs j, parties p, parties_joueurs__joueurs_parties q, partie_base m " \
    "where s.isTest = 0 " \
    "and j.session_id = s.id " \
    "and q.joueurs_uid = j.uid " \
    "and p.id = q.parties_id " \
    "and m.partie_id = p.id " \
    "order by s.nom", 
    database
)
base.head()

,session,joueur,partie_id,simulation,automatique,fautesComprehension,paiementFinal,commentaires
0,201502200955,201502200955_j_0,29,0,0,0,13.544250,None
1,201502200955,201502200955_j_1,31,0,0,1,13.064250,les instructions ne sont pas trés clair. de mê...
2,201502200955,201502200955_j_2,33,0,0,0,13.467750,None
3,201502200955,201502200955_j_3,35,0,0,0,13.981875,
4,201502200955,201502200955_j_4,37,0,0,0,13.647750,


In [7]:
oneshotparts = ["revVolInfoCprOS", "questionnaireFinal"]
repeatedparts = ["revVolInfoCprREP"]

parts_all = oneshotparts + repeatedparts
partsdata = dict()
for p in parts_all:

    # one-shot part
    if p in oneshotparts:
        req = \
            "SELECT s.nom as session, j.uid as joueur, m.* " \
            "from sessions s, joueurs j, parties p, " \
            "parties_joueurs__joueurs_parties q, partie_{} m " \
            "where s.isTest = 0 " \
            "and j.session_id = s.id " \
            "and q.joueurs_uid = j.uid " \
            "and p.id = q.parties_id " \
            "and m.partie_id = p.id " \
            "order by s.nom".format(p)

    # repeated part
    elif p in repeatedparts:
        req = \
            "SELECT s.nom as session, j.uid as joueur, r.* " \
            "from sessions s, joueurs j, parties p, " \
            "parties_joueurs__joueurs_parties q, partie_{0} m, " \
            "partie_{0}_repetitions r " \
            "where s.isTest = 0 " \
            "and j.session_id = s.id " \
            "and q.joueurs_uid = j.uid " \
            "and p.id = q.parties_id " \
            "and m.partie_id = p.id " \
            "and r.partie_partie_id = m.partie_id " \
            "order by s.nom".format(p)

    partsdata[p] = pd.read_sql(req, database)

In [8]:
partsdata["revVolInfoCprOS"].head()

,session,joueur,partie_id,RVIOS_traitement,RVIOS_groupe,RVIOS_groupe_place,RVIOS_groupe_identifiant,RVIOS_periode,RVIOS_decision,RVIOS_information,RVIOS_information_valeur,RVIOS_temps_decision,RVIOS_decision_groupe,RVIOS_gain_ecus
0,201502200955,201502200955_j_19,69,0,201502200955_g_4,1,B,0,10,NaN,NaN,10,34,38.32500
1,201502200955,201502200955_j_12,70,0,201502200955_g_1,0,A,0,10,NaN,NaN,10,31,41.98125
2,201502200955,201502200955_j_17,71,0,201502200955_g_3,0,A,0,6,NaN,NaN,10,36,23.70000
3,201502200955,201502200955_j_18,72,0,201502200955_g_0,2,C,0,6,NaN,NaN,31,30,31.12500
4,201502200955,201502200955_j_15,73,0,201502200955_g_3,2,C,0,10,NaN,NaN,31,36,35.70000


In [9]:
partsdata["questionnaireFinal"].head()

,session,joueur,partie_id,naissance,genre,nationalite,couple,etudiant,etudiant_discipline,etudiant_niveau,experiences,fratrie_nombre,fratrie_rang,sportif,sportif_type,sportif_competition,religion_place,religion_croyance,religion_nom
0,201502200955,201502200955_j_0,30,1994,1,36,0,1,1,2,1,2,2,0,NaN,NaN,1,1,4
1,201502200955,201502200955_j_1,32,1989,0,36,0,1,5,7,1,0,0,1,0,0,4,4,7
2,201502200955,201502200955_j_2,34,1992,0,36,0,1,15,5,1,1,2,1,1,0,4,3,2
3,201502200955,201502200955_j_3,36,1992,1,36,0,1,9,2,1,3,3,1,0,0,3,3,7
4,201502200955,201502200955_j_4,38,1994,0,36,0,1,5,3,1,2,1,0,NaN,NaN,3,3,4


In [11]:
partsdata["revVolInfoCprREP"].head()

,session,joueur,id,partie_partie_id,RVIREP_traitement,RVIREP_groupe,RVIREP_groupe_place,RVIREP_groupe_identifiant,RVIREP_periode,RVIREP_decision,RVIREP_information,RVIREP_information_valeur,RVIREP_temps_decision,RVIREP_decision_groupe,RVIREP_gain_periode,RVIREP_gain_cumule
0,201502200955,201502200955_j_19,81,89,0,201502200955_g_4,1,B,1,10,NaN,NaN,29,33,39.58125,39.58125
1,201502200955,201502200955_j_19,101,89,0,201502200955_g_4,1,B,2,10,NaN,NaN,10,33,39.58125,79.16250
2,201502200955,201502200955_j_19,121,89,0,201502200955_g_4,1,B,3,10,NaN,NaN,26,35,37.03125,116.19375
3,201502200955,201502200955_j_19,141,89,0,201502200955_g_4,1,B,4,10,NaN,NaN,6,35,37.03125,153.22500
4,201502200955,201502200955_j_19,161,89,0,201502200955_g_4,1,B,5,10,NaN,NaN,7,40,30.00000,183.22500


In [41]:
data = pd.merge(partsdata["revVolInfoCprREP"], partsdata["revVolInfoCprOS"], on=["session", "joueur"])
data = pd.merge(data, partsdata["questionnaireFinal"], on=["session", "joueur"])

In [44]:
data

,session,joueur,id,partie_partie_id,RVIREP_traitement,RVIREP_groupe,RVIREP_groupe_place,RVIREP_groupe_identifiant,RVIREP_periode,RVIREP_decision,RVIREP_information,RVIREP_information_valeur,RVIREP_temps_decision,RVIREP_decision_groupe,RVIREP_gain_periode,RVIREP_gain_cumule,partie_id_x,RVIOS_traitement,RVIOS_groupe,RVIOS_groupe_place,
0,201502200955,201502200955_j_19,81,89,0,201502200955_g_4,1,B,1,10,NaN,NaN,29,33,39.58125,39.58125,69,0,201502200955_g_4,1,...
1,201502200955,201502200955_j_19,101,89,0,201502200955_g_4,1,B,2,10,NaN,NaN,10,33,39.58125,79.16250,69,0,201502200955_g_4,1,...
2,201502200955,201502200955_j_19,121,89,0,201502200955_g_4,1,B,3,10,NaN,NaN,26,35,37.03125,116.19375,69,0,201502200955_g_4,1,...
3,201502200955,201502200955_j_19,141,89,0,201502200955_g_4,1,B,4,10,NaN,NaN,6,35,37.03125,153.22500,69,0,201502200955_g_4,1,...
4,201502200955,201502200955_j_19,161,89,0,201502200955_g_4,1,B,5,10,NaN,NaN,7,40,30.00000,183.22500,69,0,201502200955_g_4,1,...
5,201502200955,201502200955_j_19,181,89,0,201502200955_g_4,1,B,6,10,NaN,NaN,7,35,37.03125,220.25625,69,0,201502200955_g_4,1,...
6,201502200955,201502200955_j_19,201,89,0,201502200955_g_4,1,B,7,10,NaN,NaN,7,30,43.12500,263.38125,69,0,201502200955_g_4,1,...
7,201502200955,201502200955_j_19,221,89,0,201502200955_g_4,1,B,8,10,NaN,NaN,8,34,38.32500,301.70625,69,0,201502200955_g_4,1,...
8,201502200955,201502200955_j_19,241,89,0,201502200955_g_4,1,B,9,10,NaN,NaN,10,40,30.00000,331.70625,69,0,201502200955_g_4,1,...
9,201502200955,201502200955_j_19,261,89,0,201502200955_g_4,1,B,10,10,NaN,NaN,9,39,31.48125,363.18750,69,0,201502200955_g_4,1,...


In [47]:
data.to_csv("/home/irtimid/temp/cpri.csv", sep=";", encoding="utf-8", index=False, na_rep=None)

In [48]:
len(partsdata)

3